In [16]:
import tensorflow as tf
import pandas as pd
from os import walk as walk


In [17]:
PATH_DATA = './ImageSegData/'
DIR_LABELS = 'labels'
DIR_IMAGES = 'images'

PATH_LABELS = PATH_DATA+DIR_LABELS
PATH_IMAGES = PATH_DATA+DIR_IMAGES


# Create CSV


In [35]:
class SegNet:
    def __init__(self):
        self.num_classes = 6
        self.input_width = 640
        self.input_height = 480
        self.input_channels = 3
        self._build_model()
    
    def _build_model(self):
        self.inputs_pl = tf.placeholder(tf.float32, [None, self.input_height, self.input_width, self.input_channels])
        self.labels_pl = tf.placeholder(tf.float32, [None, self.input_height, self.input_width, self.input_channels])

        # Encoder pass
        # TODO
        
        # Decoder pass
        # TODO
        
    def train(self, max_steps = 11, batch_size=3):
        
        labels = []
        for (dirpath, dirnames, filenames) in walk(PATH_LABELS):
            for name in filenames:
                path = dirpath+'/'+name
                labels.append(path)
        
        images = []
        for (dirpath, dirnames, filenames) in walk(PATH_IMAGES):
            for name in filenames:
                path = dirpath+'/'+name
                images.append(path)
                
        # Create a dataset (image, label)-pairs
        ds = tf.data.Dataset.from_tensor_slices((images, labels))
        
        def load_and_preprocess_from_paths(image_path, label_path):
            image_raw = tf.read_file(image_path)
            label_raw = tf.read_file(label_path)
            
            # TODO: PREPROCESSING (Normalization..)
            image = tf.image.decode_png(image_raw, channels=3)
            label = tf.image.decode_png(label_raw, channels=3) 
            return image, label
        
        image_label_ds = ds.map(load_and_preprocess_from_paths)
        image_label_ds = image_label_ds.shuffle(buffer_size=3)
        image_label_ds = image_label_ds.repeat()
        image_label_ds = image_label_ds.batch(batch_size)
        #image_label_ds = image_label_ds.prefetch(buffer_size=AUTOTUNE)
        
        # create general iterator
        iterator = tf.data.Iterator.from_structure(image_label_ds.output_types,
                                               image_label_ds.output_shapes)
        
        next_element = iterator.get_next()
        training_init_op = iterator.make_initializer(image_label_ds)
        
        with tf.Session() as session:
            session.run(tf.local_variables_initializer())
            session.run(tf.global_variables_initializer())
            session.run(training_init_op)
            
            for step in range(max_steps):
                feed_dict = {self.inputs_pl: next_element}
                print(session.run(next_element))
                
                

In [36]:
model = SegNet()

In [37]:
model.train(2)

(array([[[[115, 209, 255],
         [115, 209, 255],
         [115, 209, 255],
         ...,
         [115, 209, 255],
         [115, 209, 255],
         [115, 209, 255]],

        [[115, 209, 255],
         [115, 209, 255],
         [115, 209, 255],
         ...,
         [115, 209, 255],
         [115, 209, 255],
         [115, 209, 255]],

        [[115, 209, 255],
         [115, 209, 255],
         [115, 209, 255],
         ...,
         [115, 209, 255],
         [115, 209, 255],
         [115, 209, 255]],

        ...,

        [[ 62,  63,  57],
         [ 61,  62,  57],
         [ 61,  62,  57],
         ...,
         [ 54,  55,  51],
         [ 53,  55,  51],
         [ 53,  54,  51]],

        [[ 62,  62,  57],
         [ 61,  62,  57],
         [ 61,  62,  56],
         ...,
         [ 54,  56,  52],
         [ 54,  55,  52],
         [ 54,  55,  51]],

        [[ 62,  62,  57],
         [ 61,  62,  56],
         [ 61,  62,  56],
         ...,
         [ 55,  57,  53],
       